## Imports

In [1]:
import torch
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline 
# %matplotlib inline to creating seperate window for matplotlib figures it will draw it in cell output instead

## Read Dataset

In [2]:
words = open("/kaggle/input/eureka-labs/names.txt","r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

## char to id

In [4]:
chars     = sorted(list(set(''.join(words))))
stoi      = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos      = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Dataset

In [5]:
def build_dataset(words):
    block_size = 3 # context length : how many characters do we take to predict next one
    X, Y       = [],[]

    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '----->', itos[ix])
            context = context[1:] + [ix] #crop and apppend

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return X,Y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr   = build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xte, Yte   = build_dataset(words[n2])

## Initialize weights

In [6]:
#utility function we are going to use later for comparing manual gradients with PyTorch gradients
def cmp(s, dt, t):
    torch.all(dt == t.grad).item()
    app     = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).asb().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [7]:
n_hidden   = 64 # the number of neurons in the hidden layer of the MLP
vocab_size = 27
n_embd     = 10 # the dimensionality of the character embedding vectors
block_size = 3

g = torch.Generator().manual_seed(2147483647) # for reproducability
C = torch.randn((vocab_size, n_embd), generator=g)

W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size) ** 0.5) #kamming dist 
b1 = torch.randn((n_hidden), generator=g) * 0.1 #for making initial weights near to zero to mask un uinform distribution

W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn((vocab_size), generator=g) * 0.1

bngain = torch.ones((1,n_hidden)) * 0.1 + 0.1   #Batch normalization trainable parameter
bnbias = torch.zeros((1,n_hidden)) * 0.1  #Batch normalization trainable parameter

bnmean_running = torch.zeros((1,n_hidden))
bnstd_running  = torch.ones((1,n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

## Training loop

In [8]:
batch_size = 32
n = batch_size
# minibatch
ix     = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [9]:
#forward pass "chunkated" into smaller steps that are possible to backward one at a time
emb     = C[Xb] #embed the characters into the vectors
embcat  = emb.view(emb.shape[0], -1) # concate the vectors

#Linear layer 1
hprebn  = embcat @ W1 + b1 #hidden layer pre activation

#------------------------------- BatchNorm layer -------------------------------
bnmeani   = 1/n*hprebn.sum(0, keepdim=True)
bndiff    = hprebn - bnmeani
bndiff2   = bndiff**2
bnvar     = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5 #??
bnraw     = bndiff * bnvar_inv
hpreact   = bngain * bnraw + bnbias
#------------------------------------------------------------------------------
#Non linearity
h       = torch.tanh(hpreact) #hidden layer

#Linear layer 2
logits  = h @ W2 + b2 # output layer

#-----------------------------loss function------------------------------------------------
logit_maxes    = logits.max(1, keepdim=True).values
norm_logits    = logits - logit_maxes # subtract max for numerical stability
counts         = norm_logits.exp()
counts_sum     = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs          = counts * counts_sum_inv
logprobs       = probs.log()
loss           = -logprobs[range(n), Yb].mean()
#-----------------------------------------------------------------------------

#Backward pass
for p in parameters:
    p.grad = None
    
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.2848, grad_fn=<NegBackward0>)